<a href="https://colab.research.google.com/github/xbyork/EscapeRoom/blob/main/OpenAI_Connection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧩 Cyber Escape Room Generator (OpenAI + CSV RAG) — No UI Version

This notebook builds a **single self-contained HTML** escape-room game from your **CSV standards** using **OpenAI** with a minimal RAG step.

**Usage (no widgets):**
1. Put your `standards.csv` (columns: `standard_id, grade_band, topic, standard_text`) in `/content/` **or** upload when prompted.
2. The **grade, topics, # puzzles, difficulty, subject** are **hard-coded** below for testing.
3. The generated HTML is saved in `/content` and **optionally** copied to Google Drive.


In [1]:
!pip -q install --upgrade openai pandas python-dotenv
import os, re, glob, shutil, pathlib, textwrap
import pandas as pd
from dotenv import load_dotenv
load_dotenv()  # If you uploaded a .env with OPENAI_API_KEY

# === Configuration (edit as needed) ===
GRADE       = 'middle'
TOPICS      = 'phishing,passwords'
PUZZLES     = 3
DIFFICULTY  = 'easy'
SUBJECT     = 'cybersecurity'
MODEL_NAME  = 'gpt-5-chat-latest'
SAVE_TO_DRIVE = True  # set False to skip Drive copy
DRIVE_DIR = '/content/drive/MyDrive/Colab Notebooks'  # where to copy in Drive

# API key: from env var FIRST; if not present, prompt once.
OPENAI_API_KEY = os.getenv('prompt101')
if not OPENAI_API_KEY:
    try:
        OPENAI_API_KEY = input('prompt101').strip()
    except EOFError:
        OPENAI_API_KEY = ''
if not OPENAI_API_KEY:
    raise ValueError('Missing OPENAI_API_KEY')

from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)
print('✅ OpenAI client ready.')

prompt101sk-proj-pyDhKxQsNf3KHE4FN2jvT3BlbkFJPRh0tlXaipEZl2OgHGbV
✅ OpenAI client ready.


In [2]:
# === Load standards.csv (from /content or upload) ===
from google.colab import files

csv_path = '/content/standards.csv'
if not os.path.isfile(csv_path):
    print('Upload your standards CSV (e.g., standards.csv) ...')
    uploaded = files.upload()
    if not uploaded:
        raise ValueError('No CSV uploaded.')
    csv_path = list(uploaded.keys())[0]

df = pd.read_csv(csv_path)
print('📄 Loaded:', csv_path, '| Rows:', len(df))
display(df.head(8))

📄 Loaded: /content/standards.csv | Rows: 10


,standard_id,grade_band,topic,standard_text
0,CSDF-1.1,elementary,phishing,Recognize when an online message looks suspici...
1,CSDF-1.2,elementary,passwords,Explain why you should not share your password...
2,CSDF-2.1,middle,phishing,Identify suspicious emails and messages that m...
3,CSDF-2.2,middle,passwords,"Demonstrate creation of strong, unique passwords"
4,CSDF-2.3,middle,networks,Describe what a firewall does to protect devic...
5,NICE-K12-PRIV-1,high,privacy,Explain the importance of protecting personall...
6,NICE-K12-NET-2,high,networks,Compare how different types of network defense...
7,NICE-K12-PHISH-3,high,phishing,Analyze real-world phishing examples and expla...


In [3]:
# === Core generation (no UI; selections are hard-coded above) ===

def tiny_retrieve(df: pd.DataFrame, grade_band: str, topics: str):
    topics_list = [t.strip().lower() for t in topics.split(',') if t.strip()]
    mask = (df['grade_band'].str.lower() == grade_band.lower()) & (df['topic'].str.lower().isin(topics_list))
    return df[mask].copy(), topics_list

SYSTEM_PROMPT = '''\
You are the Escape Room Game Master & Curriculum Aligner. Use retrieved standards (CSV lines provided) to design K–12 cybersecurity escape-room puzzles.

How You Work:
1) Parse teacher input (grade_band, subject, topic(s), difficulty, #puzzles).
2) Retrieve (RAG): use ONLY the provided rows. Do not show standards text to players.
3) Plan a short narrative + list of puzzle objectives mapped to the retrieved standards.
4) Generate puzzles (MCQ/SHORT/MATCH) with: question, choices (if MCQ), correct answer(s), explanation (why it matters), and 3 hints.
5) Present sequentially in an escape-room storyline (locks/doors/codes) with feedback: **Correct!/Incorrect!**, brief reason, game effect, and HINT/SCORE/SKIP controls.
6) Safety: no hacking/malware/exploit instructions; keep age-appropriate; school/home/device contexts.

Output Requirements:
- Produce EXACTLY ONE self-contained HTML file (inline CSS/JS, no external CDNs).
- Include narrative intro; sequential puzzles; hint/skip/score; lock-opening feedback.
- Include a persistent “Download This Game (.html)” button that downloads the entire current page using Blob + outerHTML.
- Set window.__gameGrade and window.__gameTopics for filename.

Do not output any extra commentary—HTML only.
'''

def build_user_message(grade, subject, topics, puzzles, difficulty, retrieved_df):
    teacher_input = f"grade={grade}; subject={subject}; topic={topics}; puzzles={puzzles}; difficulty={difficulty}"
    retrieved_block = "\n".join(
        f"{r.standard_id}\t{r.grade_band}\t{r.topic}\t{r.standard_text}"
        for r in retrieved_df.itertuples(index=False)
    )
    return f'''\
Teacher input:
{teacher_input}

Use ONLY these retrieved rows to ground the puzzles (do not display them to players):
RETRIEVED_STANDARDS_START
{retrieved_block}
RETRIEVED_STANDARDS_END

Output: Exactly ONE complete self-contained HTML/JS file with a built-in "Download This Game (.html)" button.
'''

def call_openai_html(api_key, system_prompt, user_message, model=MODEL_NAME):
    _client = OpenAI(api_key=api_key)
    resp = _client.responses.create(
        model=model,
        input=[
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": user_message}
        ]
    )
    html = getattr(resp, 'output_text', None)
    if not html:
        # Fallback for older response shapes
        try:
            html = resp.choices[0].message.content
        except Exception as e:
            raise RuntimeError(f'Could not read response text: {e}')
    return html

# RAG step
retrieved_df, topics_list = tiny_retrieve(df, GRADE, TOPICS)
if retrieved_df.empty:
    raise ValueError('No matching standards found. Adjust GRADE/TOPICS to match your CSV.')

# Build messages and call model
user_msg = build_user_message(GRADE, SUBJECT, TOPICS, PUZZLES, DIFFICULTY, retrieved_df)
print('⏳ Generating HTML...')
html = call_openai_html(OPENAI_API_KEY, SYSTEM_PROMPT, user_msg)

# Save to /content
safe_topics = re.sub(r'[^A-Za-z0-9]+', '_', TOPICS).strip('_').lower() or 'cyber'
out_name = f"cyber_escape_{GRADE}_{safe_topics}.html"
out_path = f"/content/{out_name}"
with open(out_path, 'w', encoding='utf-8') as f:
    f.write(html)
print('💾 Saved:', out_path)

# Export a global so other cells/scripts can find the latest artifact
globals()['LAST_HTML_PATH'] = out_path
print('LAST_HTML_PATH =', LAST_HTML_PATH)

# Optional: copy to Google Drive
if SAVE_TO_DRIVE:
    try:
        from google.colab import drive
        if not os.path.ismount('/content/drive'):
            drive.mount('/content/drive')
        pathlib.Path(DRIVE_DIR).mkdir(parents=True, exist_ok=True)
        dest = os.path.join(DRIVE_DIR, os.path.basename(out_path))
        shutil.copy2(out_path, dest)
        print('✅ Also saved to Google Drive:', dest)
    except Exception as e:
        print('⚠️ Drive copy skipped/failed:', e)

⏳ Generating HTML...
💾 Saved: /content/cyber_escape_middle_phishing_passwords.html
LAST_HTML_PATH = /content/cyber_escape_middle_phishing_passwords.html
✅ Also saved to Google Drive: /content/drive/MyDrive/Colab Notebooks/cyber_escape_middle_phishing_passwords.html


In [10]:
# (Optional) Standalone: copy the newest generated HTML to Drive again
import os, glob, shutil, pathlib
from google.colab import drive

candidate = globals().get('LAST_HTML_PATH', '')
if not candidate or not os.path.isfile(candidate):
    htmls = sorted(glob.glob('/content/*.html'), key=os.path.getmtime, reverse=True)
    candidate = htmls[0] if htmls else ''

if not candidate:
    raise FileNotFoundError("No HTML file found. Run the generate cell first.")

if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')
pathlib.Path(DRIVE_DIR).mkdir(parents=True, exist_ok=True)
dest = os.path.join(DRIVE_DIR, os.path.basename(candidate))
shutil.copy2(candidate, dest)
print('📤 Copied to Drive:', dest)


📤 Copied to Drive: /content/drive/MyDrive/Colab Notebooks/cyber_escape_middle_phishing_passwords.html
